# Imbalanced-learn for

In [1]:
from sklearn.metrics import classification_report,confusion_matrix,log_loss,auc,plot_confusion_matrix
from sklearn.preprocessing import (
    MinMaxScaler, label_binarize, OneHotEncoder, LabelEncoder)
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from itertools import cycle, product
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
import seaborn as sns
import warnings
import random
from tqdm import tqdm_notebook
warnings.filterwarnings('ignore')
%matplotlib inline

## Raw Data

In [2]:
dataset=pd.read_excel('../Case.xlsx',sheet_name='initial')
data=dataset.values

#array=[]
#for i in data:
#    if i[0]==2:
#        array.append(list(i))
#    data=np.array(array)
    
X =data[:,1:18]
Y =data[:,19]
seed = random.randint(0,1000)
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

dict_resample = {}
dict_model = {}

## Over-sampling

### RandomOverSample

In [3]:
# 简单的复制样本
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled ,y_resampled= ros.fit_resample(X_train,y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
dict_resample['RandomOverSampler'] = (X_resampled ,y_resampled)

[(0.0, 1795), (1.0, 1795)]


### SMOTE

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTE'] = (X_resampled ,y_resampled)

[(0.0, 1795), (1.0, 1795)]


### ADASYN

In [5]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['ADASYN'] = (X_resampled ,y_resampled)

[(0.0, 1795), (1.0, 1756)]


### BorderlineSMOTE

In [6]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-1').fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['BorderlineSMOTE-1'] = (X_resampled ,y_resampled)

[(0.0, 1795), (1.0, 1795)]


In [7]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-2').fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['BorderlineSMOTE-2'] = (X_resampled ,y_resampled)

[(0.0, 1795), (1.0, 1794)]


## Over- and under-sampling

### SMOTEENN

In [8]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTEENN'] = (X_resampled ,y_resampled)

[(0.0, 1097), (1.0, 1614)]


### SMOTETomek

In [9]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTETomek'] = (X_resampled ,y_resampled)

[(0.0, 1760), (1.0, 1760)]


## Bagging

使用训练数据的不同随机子集来训练每个 Base Model，最后每个 Base Model 权重相同，分类问题进行投票，回归问题平均。

随机森林就用到了Bagging，并且具有天然的并行性。

### Bagging

In [10]:
print(sorted(Counter(y_train).items()))
dict_model['Bagging'] = (X_train ,y_train)

[(0.0, 1795), (1.0, 157)]


### BalancedBagging

In [11]:
print(sorted(Counter(y_train).items()))
dict_model['BalancedBagging'] = (X_train ,y_train)

[(0.0, 1795), (1.0, 157)]


### BalancedRandomForest

In [12]:
print(sorted(Counter(y_train).items()))
dict_model['BalancedRandomForest'] = (X_train ,y_train)

[(0.0, 1795), (1.0, 157)]


## Boosting

Boosting是一种迭代的方法，每一次训练会更关心上一次被分错的样本，比如改变被错分的样本的权重的Adaboost方法。还有许多都是基于这种思想，比如Gradient Boosting等。

### RUSBoost

In [13]:
print(sorted(Counter(y_train).items()))
dict_model['RUSBoost'] = (X_train ,y_train)

[(0.0, 1795), (1.0, 157)]


### EasyEnsemble

In [14]:
print(sorted(Counter(y_train).items()))
dict_model['EasyEnsemble'] = (X_train ,y_train)

[(0.0, 1795), (1.0, 157)]


## 评估

In [15]:
from sklearn.svm import LinearSVC
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
################### train #####################################
scores={}
cm = []
auc = []
for name, (X, y) in tqdm_notebook(list(dict_model.items())):
    if name in ['Bagging']:
        from sklearn.ensemble import BaggingClassifier
        from sklearn.tree import DecisionTreeClassifier
        clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                random_state=0)
    elif name in ['BalancedBagging']:
        from imblearn.ensemble import BalancedBaggingClassifier
        clf = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                        sampling_strategy='auto',
                                        replacement=False,
                                        random_state=0)
    elif name in ['BalancedRandomForest']:
        from imblearn.ensemble import BalancedRandomForestClassifier
        clf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
    elif name in ['RUSBoost']:
        from imblearn.ensemble import RUSBoostClassifier
        clf = RUSBoostClassifier(random_state=0)
    elif name in ['EasyEnsemble']:
        from imblearn.ensemble import EasyEnsembleClassifier
        clf = EasyEnsembleClassifier(random_state=0)

    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score

    # 混淆矩阵
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append((name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    RANDOM_STATE = 2019
    clf = RandomForestClassifier(n_estimators=161,
                                     max_depth=49,
                                     max_features="sqrt",
                                     random_state=RANDOM_STATE)
    
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('RF'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
from sklearn.linear_model import LogisticRegression
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    clf = LogisticRegression(penalty="l2",solver="liblinear",C=0.8,max_iter=1000)
       
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('LR'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    clf = AdaBoostClassifier(n_estimators=161,
                             learning_rate=0.5,
                                     random_state=RANDOM_STATE)
    
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('AdaBoost'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
for i in auc:
    print("{:.3f}".format(i))

0.758
0.774
0.813
0.771
0.766
0.792
0.798
0.801
0.803
0.782
0.774
0.797
0.834
0.829
0.830
0.830
0.823
0.830
0.834
0.801
0.776
0.784
0.766
0.764
0.779
0.782


In [20]:
for name,c in cm:
    print("{:.3f}".format(c[1][1]/(c[1][1]+c[0][1]+c[1][0])))

0.056
0.167
0.149
0.155
0.140
0.111
0.198
0.187
0.163
0.158
0.163
0.187
0.175
0.175
0.180
0.191
0.161
0.144
0.179
0.187
0.159
0.161
0.157
0.156
0.141
0.152


In [21]:
for name,c in cm:
    print("{:}".format(name))

Bagging
BalancedBagging
BalancedRandomForest
RUSBoost
EasyEnsemble
RFRandomOverSampler
RFSMOTE
RFADASYN
RFBorderlineSMOTE-1
RFBorderlineSMOTE-2
RFSMOTEENN
RFSMOTETomek
LRRandomOverSampler
LRSMOTE
LRADASYN
LRBorderlineSMOTE-1
LRBorderlineSMOTE-2
LRSMOTEENN
LRSMOTETomek
AdaBoostRandomOverSampler
AdaBoostSMOTE
AdaBoostADASYN
AdaBoostBorderlineSMOTE-1
AdaBoostBorderlineSMOTE-2
AdaBoostSMOTEENN
AdaBoostSMOTETomek


In [22]:
for name,c in cm:
    print("{}: {:.3f}".format(name,c[1][1]/(c[1][1]+c[0][1]+c[1][0])))

Bagging: 0.056
BalancedBagging: 0.167
BalancedRandomForest: 0.149
RUSBoost: 0.155
EasyEnsemble: 0.140
RFRandomOverSampler: 0.111
RFSMOTE: 0.198
RFADASYN: 0.187
RFBorderlineSMOTE-1: 0.163
RFBorderlineSMOTE-2: 0.158
RFSMOTEENN: 0.163
RFSMOTETomek: 0.187
LRRandomOverSampler: 0.175
LRSMOTE: 0.175
LRADASYN: 0.180
LRBorderlineSMOTE-1: 0.191
LRBorderlineSMOTE-2: 0.161
LRSMOTEENN: 0.144
LRSMOTETomek: 0.179
AdaBoostRandomOverSampler: 0.187
AdaBoostSMOTE: 0.159
AdaBoostADASYN: 0.161
AdaBoostBorderlineSMOTE-1: 0.157
AdaBoostBorderlineSMOTE-2: 0.156
AdaBoostSMOTEENN: 0.141
AdaBoostSMOTETomek: 0.152
